# Задание

Обучить decoder-only модель предсказывать ответ системы на запрос пользователя.

1. Проверить работу модели в сценарии близком к выбранному датасету в zero-shot режиме.
2. Посчитать метрики bleu и meteor исходной модели на валидационном сабсете выбранного датасета.
3. Дообучить модель на обучающем сабсете выбранного датасета.
4. Проверить работу модели в сценарии близком к выбранному датасету.
5. Посчитать метрики bleu и meteor обученной модели на валидационном сабсете выбранного датасета.

In [54]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [55]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

*По идеи должно улучшить обучение*

In [56]:
special_tokens_dict = {
    "additional_special_tokens": ["<user>", "<bot>"]
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

# Тестовый запуск

In [57]:
text = 'Привет! Расскажи, как твои дела?'

In [58]:
inputs = tokenizer(text, return_tensors='pt')

In [59]:
with torch.no_grad():
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,
        max_length=32,
    )

In [60]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Привет! Расскажи, как твои дела?
А ты откуда знаешь что у меня дома кто-то есть. Что я на даче живу??) Приветик))
Привет! Расскажи, как твои дела?
Я не могу. И я с тобой тоже... но пока ничего плохого сказать тебе нельзя: в последнее время ты стала
Привет! Расскажи, как твои дела?
Смотря что за человек. Если не секрет - сколько ему лет и где он живет в Москве (может с родителями


# Fine-Tuning

In [61]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

In [62]:
df = load_dataset("zjkarina/matreshka")
train = df['train'].to_pandas()
test, valid = train_test_split(df['validation'].to_pandas(), test_size=0.5)

**Вспомогательные функции**

In [63]:
def _df_to_gpt_format(df):

    dialogs = []

    for _, row in df.iterrows():
        role = row['role']
        dialog = row['dialog']

        if role is None or dialog is None or len(role) < 2:
            continue

        role = [r.lower() for r in role]

        if any(r not in ['user', 'bot'] for r in role):
            continue

        role, dialog = _drop_first_bot_last_user_utterings(role, dialog)

        if len(role) < 2:
            continue

        role, dialog = _shrink_roles_and_dialogue(role, dialog)

        # формат для GPT
        text = ''
        for r, d in zip(role, dialog):
            prefix = '<user>' if r == 'user' else '<bot>'
            text += f'{prefix}: {d.strip()}\n'

        dialogs.append(text.strip() + " </s>")

    return dialogs

In [64]:
def _shrink_roles_and_dialogue(row_role, row_dialog):
    shrinked_role = []
    shrinked_dialog = []
    prev_role = ''
    for i in range(len(row_dialog)):
        if row_role[i] != prev_role:
            shrinked_role.append(row_role[i])
            shrinked_dialog.append(row_dialog[i])
        else:
            shrinked_dialog[-1] += ' ' + row_dialog[i]
        prev_role = row_role[i]

    assert len(shrinked_dialog) == len(shrinked_role)

    shrinked_role, shrinked_dialog = _drop_first_bot_last_user_utterings(shrinked_role, shrinked_dialog)

    return shrinked_role, shrinked_dialog

In [65]:
def _drop_first_bot_last_user_utterings(role, dialog):
    if role[-1] == 'user':
        role, dialog = role[:-1], dialog[:-1]
    if role[0] == 'bot':
        role, dialog = role[1:], dialog[1:]
    return role, dialog

In [66]:
format_train = _df_to_gpt_format(train)
format_val = _df_to_gpt_format(valid)
format_test = _df_to_gpt_format(test)

In [70]:
df_train = pd.DataFrame(format_train, columns=['text'])
df_val = pd.DataFrame(format_val, columns=['text'])
df_test = pd.DataFrame(format_test, columns=['text'])

**Токенизация для GPT**

In [81]:
def tokenize_data(df, tokenizer, max_length=128):
    tokens = tokenizer.batch_encode_plus(
        df["text"].tolist(),
        max_length=max_length,
        truncation=True,
        padding="max_length",   # или "longest" если не хочешь фиксированный размер
        return_tensors="pt"
    )

    labels = tokens['input_ids'].clone()
    labels[labels == tokenizer.pad_token_id] = -100
    tokens['labels'] = labels

    return tokens

In [82]:
train_tokens = tokenize_data(df_train, tokenizer)
val_tokens   = tokenize_data(df_val, tokenizer)
test_tokens  = tokenize_data(df_test, tokenizer)

In [83]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(
    train_tokens['input_ids'],
    train_tokens['attention_mask'],
    train_tokens['labels']
)

val_dataset = TensorDataset(
    val_tokens['input_ids'],
    val_tokens['attention_mask'],
    val_tokens['labels']
)

test_dataset = TensorDataset(
  test_tokens['input_ids'],
  test_tokens['attention_mask'],
  test_tokens['labels']
)

In [84]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False)

In [85]:
device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [86]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

In [87]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

**Функции обучения**

In [88]:
from tqdm import tqdm

def train_epoch(model, loader, epoch, num_epochs, optimizer, mode):
    total_loss = 0
    is_train = (mode == 'Training')
    model.train() if is_train else model.eval()

    loop = tqdm(loader, desc=f"{mode} epoch {epoch+1}/{num_epochs}")

    for input_ids, attention_mask, labels in loop:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        if is_train:
            optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True,
        )

        loss = outputs.loss
        total_loss += loss.item()

        if is_train:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

    avg_loss = total_loss / len(loader)
    print(f"{mode} epoch {epoch+1}/{num_epochs}: Loss = {avg_loss:.4f}")

In [89]:
def train(
    model,
    train_loader,
    val_loader,
    optimizer,
    num_epochs=5,
):
    for epoch in range(num_epochs):
        model.train()
        train_epoch(model, train_loader, epoch, num_epochs, optimizer, mode='Training')

        model.eval()
        with torch.no_grad():
            train_epoch(model, val_loader, epoch, num_epochs, optimizer, mode='Validation')

**Обучение**

In [90]:
train(model, train_loader, val_loader, optimizer, num_epochs=5)

Training epoch 1/5: 100%|██████████| 832/832 [04:13<00:00,  3.29it/s]


Training epoch 1/5: Loss = 2.1267


Validation epoch 1/5: 100%|██████████| 104/104 [00:09<00:00, 11.29it/s]


Validation epoch 1/5: Loss = 1.7702


Training epoch 2/5: 100%|██████████| 832/832 [04:12<00:00,  3.30it/s]


...

Validation epoch 4/5: Loss = 1.6850


Training epoch 5/5: 100%|██████████| 832/832 [04:12<00:00,  3.30it/s]


Training epoch 5/5: Loss = 1.4654


Validation epoch 5/5: 100%|██████████| 104/104 [00:09<00:00, 11.31it/s]

Validation epoch 5/5: Loss = 1.6802


# Получение ответов от FT модели

In [91]:
inputs['input_ids'] = inputs['input_ids'].to(device)
inputs['attention_mask'] = inputs['attention_mask'].to(device)

In [92]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,             # sampling or greedy decoding
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,     # https://arxiv.org/pdf/1909.05858.pdf
        max_length=32,
    )

In [93]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Привет! Расскажи, как твои дела?
: Привет. У меня все хорошо и спасибо за спрос о курсах программирования для начинающих... А у тебя есть какие
Привет! Расскажи, как твои дела?
: Привет. У меня все хорошо с работой и учебой в университете Сейчас учусь на факультете журналистики В
Привет! Расскажи, как твои дела?
: Привет. У меня все хорошо а у тебя что нового интересного происходит в жизни/в работе или учебе


In [94]:
def generate(model, tokenizer, prompt, max_new_tokens=50, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )
    return [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

# Вычисление BLEU-метрик на валидационных данных

In [95]:
!pip install torchmetrics;

In [96]:
import re
from torchmetrics.text import BLEUScore

In [97]:
import re
from torchmetrics.text import BLEUScore
from transformers import AutoModelForCausalLM

bleu = BLEUScore(n_gram=1)
cnt = 100

initial_model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2").to(device)
finetuned_model = model

initial_bleu_score = 0.
avg_bleu_score = 0.

with torch.no_grad():
    for _, row in tqdm(df_val.head(cnt).iterrows(), total=cnt):
        prompt = row['text']
        target = row['text'].split('<bot>')[-1].strip()

        init_pred = generate(initial_model, tokenizer, prompt)[0]
        fine_pred = generate(finetuned_model, tokenizer, prompt)[0]

        # Обрезаем до первого предложения (по желанию)
        target_clean = re.split(r'\.|!|\?', target)[0]
        init_clean = re.split(r'\.|!|\?', init_pred)[0]
        fine_clean = re.split(r'\.|!|\?', fine_pred)[0]

        initial_bleu_score += bleu(init_clean, target_clean).item()
        avg_bleu_score += bleu(fine_clean, target_clean).item()

print('avg bleu:', avg_bleu_score * 1. / cnt)
print('avg initial bleu:',  initial_bleu_score * 1. / cnt)

100%|██████████| 100/100 [02:28<00:00,  1.49s/it]

avg bleu: 0.0976013445481658
avg initial bleu: 0.0976013445481658


# Вычисление METEOR-метрик на валидационных данных

In [98]:
!pip install evaluate;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00


In [99]:
import evaluate

meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [100]:
cnt = 100

initial_model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2").to(device)
finetuned_model = model

avg_meteor_score = 0.
initial_meteor_score = 0.

with torch.no_grad():
    for _, row in tqdm(df_val.head(cnt).iterrows(), total=cnt):
        prompt = row['text']
        target = row['text'].split('<bot>')[-1].strip()

        pred_finetuned = generate(finetuned_model, tokenizer, prompt, num_return_sequences=1)[0]
        pred_initial = generate(initial_model, tokenizer, prompt, num_return_sequences=1)[0]

        target_clean = re.split(r'\.|!|\?', target)[0]
        pred_finetuned_clean = re.split(r'\.|!|\?', pred_finetuned)[0]
        pred_initial_clean = re.split(r'\.|!|\?', pred_initial)[0]

        metric = meteor.compute(predictions=[pred_finetuned_clean], references=[target_clean])
        initial_metric = meteor.compute(predictions=[pred_initial_clean], references=[target_clean])

        avg_meteor_score += metric['meteor']
        initial_meteor_score += initial_metric['meteor']

print('avg meteor:', avg_meteor_score * 1. / cnt)
print('avg initial meteor:', initial_meteor_score * 1. / cnt)

100%|██████████| 100/100 [02:09<00:00,  1.30s/it]

avg meteor: 0.14646473518046407
avg initial meteor: 0.14693380355737207


# Выводы

1. Метрики не изменились, однако визуально, ответ после дообучения точнее.
<br>Это может быть связано с тем, что:
- метрики не видят изменений, хотя они есть.
- модель `rugpt3small` упёрлась в "потолок"
- данных недостаточно или они менее полезны
- необходимо, в сравнении с `RuT5`, дольше обучать модель (увеличить `learning rate`, `tokenizer size`, `batch_size`)
- как вариант, можно убрать токены `<user>`, `<bot>`. Возможно это запутало модель
2. Метрики очень низкого уровня
- Однако у нас не так много данных для оценки: один эталон и короткие ответы
- *Лучший результат полученный мной для `METEOR` был `0.204`, которого всё равно недостаточно*
3. Ответ после дообучения действительно похож на диалог, в отличии от первоначального ответа, однако местами присутствуют галлюцинации и нерелевантная информация
- Вариантом решения может стать применение большей модели напр. `rugpt3medium`